In [2]:
!pip install webdriver_manager

In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# ----------------------------
# Setup WebDriver
# ----------------------------
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

results = []

try:
    for question_num in range(151, 251):  # Questions 151 to 251

        search_query = f"examtopics databricks data engineer professional certification question {question_num}"
    
        # 1. Open Google
        driver.get("https://www.google.com/")
        time.sleep(5)

        # 2. Enter search query
        search_box = driver.find_element(By.NAME, "q")
        search_box.clear()
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)

        time.sleep(5)

        # 3. Capture first examtopics result
        links = driver.find_elements(By.CSS_SELECTOR, "a[jsname='UWckNb']")
        
        url = None
        for link in links:
            href = link.get_attribute("href")
            if href and href.startswith("https://www.examtopics.com/discussions/databricks/view/"):
                url = href
                break

        results.append({
            "question_num": question_num,
            "search_query": search_query,
            "url": url,
            "description": "ExamTopics link found" if url else "No matching URL found"
        })

        print(f"✔ Processed question {question_num}")

finally:
    driver.quit()

# ----------------------------
# Create DataFrame
# ----------------------------
df = pd.DataFrame(results)
print(df)


✔ Processed question 151
✔ Processed question 152
✔ Processed question 153
✔ Processed question 154
✔ Processed question 155
✔ Processed question 156
✔ Processed question 157
✔ Processed question 158
✔ Processed question 159
✔ Processed question 160
✔ Processed question 161
✔ Processed question 162
✔ Processed question 163
✔ Processed question 164
✔ Processed question 165
✔ Processed question 166
✔ Processed question 167
✔ Processed question 168
✔ Processed question 169
✔ Processed question 170
✔ Processed question 171
✔ Processed question 172
✔ Processed question 173
✔ Processed question 174
✔ Processed question 175
✔ Processed question 176
✔ Processed question 177
✔ Processed question 178
✔ Processed question 179
✔ Processed question 180
✔ Processed question 181
✔ Processed question 182
✔ Processed question 183
✔ Processed question 184
✔ Processed question 185
✔ Processed question 186
✔ Processed question 187
✔ Processed question 188
✔ Processed question 189
✔ Processed question 190


In [44]:
df

,question_num,search_query,url,description
0,151,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,ExamTopics link found
1,152,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,ExamTopics link found
2,153,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,ExamTopics link found
3,154,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,ExamTopics link found
4,155,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,ExamTopics link found
...,...,...,...,...
95,246,examtopics databricks data engineer profession...,None,No matching URL found
96,247,examtopics databricks data engineer profession...,None,No matching URL found
97,248,examtopics databricks data engineer profession...,None,No matching URL found
98,249,examtopics databricks data engineer profession...,None,No matching URL found


In [45]:

# Using the URL in the dataframe to scrape the actual question and options from examtopics.com
import requests
from bs4 import BeautifulSoup

def scrape_examtopics_question(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract question text
        question_tag = soup.find('p', class_='card-text')
        question = question_tag.get_text(strip=True) if question_tag else 'Question not found'
        
        # Extract image URLs from question
        image_urls = []
        if question_tag:
            images = question_tag.find_all('img')
            image_urls = [img.get('src') for img in images if img.get('src')]

        # Extract all options (correct and others)
        options = []
        for li in soup.find_all('li', class_=['multi-choice-item', 'multi-choice-item correct-hidden']):
            option_text = li.get_text(strip=True)
            is_correct = 'correct-hidden' in li.get('class', [])
            options.append({'option': option_text, 'is_correct': is_correct})

        return {'question': question, 'options': options, 'image_urls': image_urls}
    except Exception as e:
        return {'question': None, 'options': [], 'image_urls': [], 'error': str(e)}

# Example: Scrape the first row's URL from the dataframe
if not df.empty:
    sample_url = df.iloc[0]['url']
    result = scrape_examtopics_question(sample_url)
    print('Question:', result['question'])
    print('Options:')
    for opt in result['options']:
        print(f"  {'[Correct]' if opt['is_correct'] else '[ ]'} {opt['option']}")
    if result['image_urls']:
        print('Image URLs:')
        for img_url in result['image_urls']:
            print(f"  {img_url}")
else:
    print('DataFrame is empty, no URLs to scrape.')


Question: A junior data engineer is working to implement logic for a Lakehouse table named silver_device_recordings. The source data contains 100 unique fields in a highly nested JSON structure.The silver_device_recordings table will be used downstream to power several production monitoring dashboards and a production model. At present, 45 of the 100 fields are being used in at least one of these applications.The data engineer is trying to determine the best approach for dealing with schema declaration given the highly-nested structure of the data and the numerous fields.Which of the following accurately presents information about Delta Lake and Databricks that may impact their decision-making process?
Options:
  [ ] A.The Tungsten encoding used by Databricks is optimized for storing string data; newly-added native support for querying JSON strings means that string types are always most efficient.
  [ ] B.Because Delta Lake uses Parquet for data storage, data types can be easily evolv

In [46]:
# Iterate all URLs in the dataframe and create a new dataframe with question_num, search_query, url, Question, Options, Correct answer, and Comments
import pandas as pd

results_list = []
for idx, row in df.iterrows():
    try:
        scrape_result = scrape_examtopics_question(row['url'])
        question = scrape_result.get('question', '')
        options = [opt['option'] for opt in scrape_result.get('options', [])]
        correct_options = [opt['option'] for opt in scrape_result.get('options', []) if opt['is_correct']]
        image_urls = scrape_result.get('image_urls', [])
        
        if scrape_result.get('error'):
            comment = scrape_result['error']
        elif not question or question == 'Question not found':
            comment = 'Question not found'
        elif not options:
            comment = 'Options not found'
        else:
            comment = ''
    except Exception as e:
        question = ''
        options = []
        correct_options = []
        image_urls = []
        comment = f'Error: {str(e)}'
    results_list.append({
        'question_num': row['question_num'],
        'search_query': row['search_query'],
        'url': row['url'],
        'Question': question,
        'Options': options,
        'Correct answer': correct_options,
        'Image URLs': image_urls,
        'Comments': comment
    })

final_df = pd.DataFrame(results_list)
final_df


,question_num,search_query,url,Question,Options,Correct answer,Image URLs,Comments
0,151,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,A junior data engineer is working to implement...,[A.The Tungsten encoding used by Databricks is...,[D.Because Databricks will infer schema using ...,[],
1,152,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The data engineering team maintains the follow...,[A.A batch job will update the enriched_itemiz...,[B.The enriched_itemized_orders_by_account tab...,[https://img.examtopics.com/certified-data-eng...,
2,153,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The data engineering team is configuring envir...,"[A.All development, testing, and production co...",[B.In environments where interactive code will...,[],
3,154,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The data architect has mandated that all table...,"[A.Whenever a database is being created, make ...","[C.Whenever a table is being created, make sur...",[],
4,155,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The marketing team is looking to share data in...,[A.Create a view on the marketing table select...,[A.Create a view on the marketing table select...,[],
...,...,...,...,...,...,...,...,...
95,246,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...
96,247,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...
97,248,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...
98,249,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...


In [47]:
final_df 

,question_num,search_query,url,Question,Options,Correct answer,Image URLs,Comments
0,151,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,A junior data engineer is working to implement...,[A.The Tungsten encoding used by Databricks is...,[D.Because Databricks will infer schema using ...,[],
1,152,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The data engineering team maintains the follow...,[A.A batch job will update the enriched_itemiz...,[B.The enriched_itemized_orders_by_account tab...,[https://img.examtopics.com/certified-data-eng...,
2,153,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The data engineering team is configuring envir...,"[A.All development, testing, and production co...",[B.In environments where interactive code will...,[],
3,154,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The data architect has mandated that all table...,"[A.Whenever a database is being created, make ...","[C.Whenever a table is being created, make sur...",[],
4,155,examtopics databricks data engineer profession...,https://www.examtopics.com/discussions/databri...,The marketing team is looking to share data in...,[A.Create a view on the marketing table select...,[A.Create a view on the marketing table select...,[],
...,...,...,...,...,...,...,...,...
95,246,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...
96,247,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...
97,248,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...
98,249,examtopics databricks data engineer profession...,None,None,[],[],[],Invalid URL 'None': No scheme supplied. Perhap...


In [48]:
final_df.to_csv('databricks_data_engineer_professional_questions_151_250.csv', index=False)

In [49]:
# I need the Question and Options from the final_df in a markdown file with Question and options listed below it.
import os
import requests
from pathlib import Path
from urllib.parse import urljoin

# Create images directory if it doesn't exist
images_dir = Path('examtopics_images')
images_dir.mkdir(exist_ok=True)

with open('examtopics_questions.md', 'a', encoding='utf-8') as f:
    # Add separator between old and new content
    f.write("\n\n")
    
    for idx, row in final_df.iterrows():
        f.write(f"### Question {row['question_num']}\n\n")
        
        # Download and reference images if they exist
        if row['Image URLs'] and len(row['Image URLs']) > 0:
            for img_idx, img_url in enumerate(row['Image URLs']):
                # Determine file extension and filename first
                ext = '.png'  # default
                if '.' in img_url.split('/')[-1]:
                    ext = '.' + img_url.split('/')[-1].split('.')[-1].split('?')[0]
                
                img_filename = f"question_{row['question_num']}_img_{img_idx + 1}{ext}"
                img_path = images_dir / img_filename
                
                # Check if image already exists
                if img_path.exists():
                    # Use existing image
                    f.write(f"![Question {row['question_num']} Image {img_idx + 1}]({images_dir}/{img_filename})\n\n")
                    print(f"✓ Using existing image for question {row['question_num']}")
                else:
                    # Try to download the image
                    try:
                        # Handle relative URLs and protocol-relative URLs
                        download_url = img_url
                        if img_url.startswith('//'):
                            download_url = 'https:' + img_url
                        elif not img_url.startswith('http'):
                            download_url = urljoin('https://www.examtopics.com/', img_url)
                        
                        # Download image with headers
                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                            'Referer': 'https://www.examtopics.com/'
                        }
                        img_response = requests.get(download_url, headers=headers, timeout=10)
                        img_response.raise_for_status()
                        
                        # Update extension from content-type if needed
                        if 'content-type' in img_response.headers:
                            content_type = img_response.headers['content-type']
                            if 'jpeg' in content_type or 'jpg' in content_type:
                                ext = '.jpg'
                            elif 'png' in content_type:
                                ext = '.png'
                            elif 'gif' in content_type:
                                ext = '.gif'
                            # Update filename with correct extension
                            img_filename = f"question_{row['question_num']}_img_{img_idx + 1}{ext}"
                            img_path = images_dir / img_filename
                        
                        # Save image
                        with open(img_path, 'wb') as img_file:
                            img_file.write(img_response.content)
                        
                        # Reference image in markdown
                        f.write(f"![Question {row['question_num']} Image {img_idx + 1}]({images_dir}/{img_filename})\n\n")
                        print(f"✓ Downloaded image for question {row['question_num']}")
                        
                    except Exception as e:
                        print(f"✗ Failed to download image for question {row['question_num']}: {e}")
                        f.write(f"[Image URL: {img_url}]\n\n")
        
        f.write(f"{row['Question']}\n\n")
        
        for option in row['Options']:
            f.write(f"- {option}\n")
        
        f.write("\n---\n\n")

print("Markdown file created successfully!")


✓ Downloaded image for question 152
✓ Downloaded image for question 158
✓ Downloaded image for question 161
✓ Downloaded image for question 163
✓ Downloaded image for question 165
✓ Downloaded image for question 166
✓ Downloaded image for question 166
✓ Downloaded image for question 173
✓ Downloaded image for question 181
✓ Downloaded image for question 182
✓ Downloaded image for question 187
✓ Downloaded image for question 188
✓ Downloaded image for question 191
✓ Downloaded image for question 193
✓ Downloaded image for question 195
✓ Downloaded image for question 196
✓ Downloaded image for question 201
✓ Downloaded image for question 202
✓ Downloaded image for question 204
✓ Downloaded image for question 209
✓ Downloaded image for question 211
✓ Downloaded image for question 214
✓ Downloaded image for question 215
✓ Downloaded image for question 216
✓ Downloaded image for question 218
✓ Downloaded image for question 227
✓ Downloaded image for question 227
✓ Downloaded image for quest